In [1]:
import pandas as pd
import networkx as nx
import datetime , time
import numpy as np
import datetime, time
import matplotlib.pyplot as plt

In [2]:
def filterGraph(df, start_time, end_time):
    return df[(start_time <= df['time']) & (df['time'] <= end_time)]

In [3]:
def delete_edges (G , start_time , end_time):
    s_tstmp = convert_date(start_time)
    e_tstmp = convert_date(end_time)
    
    edges = [(u,v,att) for u,v,attr in G.edges(data=True) if e_tstmp < attr['tstmp'][0] < e_tstmp]
    
    H = nx.DiGraph()
    H.add_edges_from(edges)
    print (list(H.edges))
    return H


def convert_date(date):
    converted_date = int(time.mktime(datetime.datetime.strptime(date, "%Y/%m/%d").timetuple()))
    return converted_date

In [4]:
def isDirect (G):
    for u , v , attr in G.edges(data = True) :
        if list(G.edges(u)) != list(G.edges(v)):
            return False

    return True

In [5]:
atq = pd.read_csv('data/sx-stackoverflow-a2q.txt' , sep=' ' , names=['starting_node','end_node','time'] )
ctq = pd.read_csv('data/sx-stackoverflow-c2a.txt' , sep=' ' , names=['starting_node','end_node','time'] )
cta = pd.read_csv('data/sx-stackoverflow-c2q.txt' , sep=' ' , names=['starting_node','end_node','time'] )

In [6]:
date_start = datetime.datetime(2008, 8, 1,1,0,0)
start_time = time.mktime(date_start.timetuple())
date_end = datetime.datetime(2008, 10, 1,1,0,0)
end_time = time.mktime(date_end.timetuple())

In [7]:
atq_filtered = filterGraph(atq , start_time , end_time )
ctq_filtered = filterGraph(ctq , start_time , end_time )
cta_filtered = filterGraph(cta , start_time , end_time )

In [8]:
combined_graph = nx.DiGraph()

In [9]:
atq_graph = nx.DiGraph(None , data = True)
cta_graph = nx.DiGraph(None, data = True )
ctq_graph = nx.DiGraph(None , data = True )

In [10]:
def graphWeighter (graph, edge_from_df) :
    if graph.has_edge(edge_from_df['starting_node'] , edge_from_df['end_node']) :
        graph[edge_from_df['starting_node']][edge_from_df['end_node']]['weight'] += 1
        graph[edge_from_df['starting_node']][edge_from_df['end_node']]['tstmp'].append(edge_from_df['time'])
    else:
        graph.add_edge(edge_from_df['starting_node'] , edge_from_df['end_node'] , weight = 1 )
        graph[edge_from_df['starting_node']][edge_from_df['end_node']]['tstmp'] = [edge_from_df['time']]

In [11]:
atq_filtered.apply(lambda x :graphWeighter( atq_graph , x) , axis=1)
cta_filtered.apply(lambda x :graphWeighter( cta_graph , x) , axis=1)
ctq_filtered.apply(lambda x :graphWeighter( ctq_graph , x) , axis=1)

0           None
1           None
2           None
3           None
4           None
            ... 
24268549    None
25180945    None
25180946    None
25180947    None
25180948    None
Length: 21492, dtype: object

In [12]:
combined_graph = nx.compose_all([atq_graph , cta_graph , ctq_graph])

In [13]:
def areSame(matrix_A, matrix_B , n ) :
    for i in range(n):
        for j in range(n):
            if (matrix_A[i,j] != matrix_B[i,j]):
                return 0
    return 1

def GraphDescription (graph) :
    
    direct = True
    number_interactions = 0
    #assess if the graph is directed or not
    graph_adjm = nx.to_numpy_matrix(graph)

    #find number of users
    nodes_number = graph_adjm.shape[0]
    print("the number of users is {}".format(nodes_number))


    #If the matrix is not symmetric means that the graph is directed
    if areSame(graph_adjm, graph_adjm.transpose() , nodes_number) :
        direct = False
        print('The graph is not directed')
    else:
        direct = True
        print('The graph is directed')

    #numebr of answere/comments
    number_interactions = (graph_adjm != 0).sum()
    
    
    #Second version that scann each position of the adjucency matrix and sum all the non-zero value
    
    #for i in range(nodes_number):
    #   for j in range(nodes_number) :
    #        if graph_adjm[i,j] != 0 :
    #            number_interactions += 1
 
    print("number of question/comments is {}".format(number_interactions))
    avg_link_user = number_interactions/nodes_number
    print("number of link per user {}".format(avg_link_user))
    
    #graph density and sparsness
    if direct :
        density = 2*number_interactions / (nodes_number*(nodes_number-1))
    else :
        density = number_interactions / (nodes_number*(nodes_number-1))
    
    density_definition = 'dense' if density>0.5 else 'sparse'
    
    print('the grpah density is {} then the graph is {}'.format(density , density_definition))
    
    return (nodes_number , direct , number_interactions , avg_link_user , density , density_definition)
    

In [14]:
GraphDescription(atq_graph)

the number of users is 9837
The graph is directed
number of question/comments is 68232
number of link per user 6.936261055199756
the grpah density is 0.0014103824837738421 then the graph is sparse


(9837, True, 68232, 6.936261055199756, 0.0014103824837738421, 'sparse')

In [15]:
def Betweeness (G , i) :
    nodes_number = len(G.nodes)
    S = 0
    no_dir = 0
    
    all_nodes = list(G.nodes)
    all_nodes.remove(i)
    
    for s in all_nodes :
        for t in all_nodes :
            paths_sti = 0
            no_dir = 0
            
            try :
                paths_st = [p for p in nx.all_shortest_paths(G , source = s , target = t ) ]
            except:
                no_dir = 1
            
            if no_dir == 0 :
                for element in paths_st :
                    paths_sti += 1 
                if paths_sti > 0 and len(paths_st) > 0 :
                    S += paths_sti/len(paths_st)
    
    try :
        all_nodes.remove(s)
    except:
        pass
    
    if S > 0 :
        return S / ((nodes_number-1) * (nodes_number-2))
    
    else :
        return (print ('Betweenes = 0'))

In [20]:
list(combined_graph.nodes).remove(8)

In [21]:
Betweeness(combined_graph , 8 )

0.5223850223850224

In [16]:
def PageRank (G , intereset_node , d = 0.85 , max_iter = 100) :
    all_nodes = list(G.nodes)
    nodes_number = len(all_nodes)
    
    
    PR_0 = {node : 1/ nodes_number -1 for node in all_nodes}
    outgoing_edges = {node : G.out_degree(node , weight = 'weight') for node in all_nodes}
    
    PR_t = PR_0.copy()
    
    for _ in range(max_iter) :
        for node in PR_t :
            in_nodes = [int(i[0]) for i in G.in_edges(node)]
            sum_of_neigh = sum([ PR_t[node] / outgoing_edges[node]  for node in in_nodes])
            
            PR_t[node] = (1-d)/nodes_number + d*sum_of_neigh
    
    print ('The page rank of node {} is {}'.format(intereset_node , PR_t[intereset_node] ))
    return PR_t[intereset_node]

In [26]:
PageRank(combined_graph , 1 )

The page rank of node 1 is 0.002636417216892069


In [17]:
def Dijkstra (G , source_node , target_node):
    dist = {node : float('inf') for node in list(G.nodes)}
    prev = {node : None for node in list(G.nodes)}
    
    unvisited_nodes = list(G.nodes)
    dist[source_node] = 0
    
    
    while unvisited_nodes :
        min_dist = float('inf')
        
        for node in unvisited_nodes :
            if dist[node] <= min_dist:
                min_dist = dist[node]
                current = node
        unvisited_nodes.remove(current)
        
        if current == target_node :
            break
        
        for neighbor in G.neighbors(current) :
            alt = dist[current] + G[current][neighbor]['weight']
            
            if alt < dist[neighbor] :
                dist[neighbor] = alt
                prev[neighbor] = current
            
        
    if prev[target_node] == None :
        return ('Disconnected',[])
        
    path = [target_node]
        
    while source_node not in path :
        path.append(prev[path[-1]])
    path.reverse()
        
    return dist[target_node],path
    

def ClosenessCentrality (G , interest_node) :
    all_nodes = list(G.nodes)
    C_interest = 0
    reachable_nodes = G.out_degree(interest_node) -1
    
    
    for node in all_nodes :
        distance_sum = 0
        if node != interest_node:
            distance,path = Dijkstra ( G , source_node = interest_node , target_node = node)
            if distance != 0 :
                C_interest += distance
                
    if C_interest > 0 :
        print('the closness centrality of node {} is {}'.format(interest_node , (len(all_nodes)-1 )/ C_interest ))
        return (len(all_nodes)-1 )/ C_interest
    else :
        print ('The closness centrality of node {} is 0'.format(interest_node))

In [32]:
ClosenessCentrality(combined_graph , 1 )

the closness centrality of node 1 is 0.5360294117647059


0.5360294117647059

In [18]:
def degree_calculator (G , node):
    if isDirect(G):
        degree = len(G.in_edges(node) + G.out_edges(node))
    else :
        degree = len(list(G.neighbors(node)))
    return degree

def DegreeCentrality (G , interest_node) :
    n_edges = len(G.edges)
    degree = degree_calculator(G , interest_node)
    degree_centrality = degree / (n_edges)
    
    print('The degree centrality of node {} is {}'.format(interest_node, degree_centrality))
    return degree_centrality

In [20]:
def BestUser (G ,  node , start_time , end_time , metric) :
    s_tstmp = convert_date(start_time)
    e_tstmp = convert_date(end_time)
    
    edges = [(s , t , attr) for s,t,attr in G.edges(data = True) if s_tstmp < attr['tstmp'][0] < e_tstmp]
    new_G = nx.DiGraph()
    new_G.add_edges_from(edges)

    if metric == 'Betweeness' :
        return Betweeness(new_G,node)
    elif metric == "PageRank" :
        return PageRank(new_G, node)
    elif metric == "ClosenessCentrality":
        return ClosenessCentrality(new_G,node)
    elif metric == "DegreeCentrality":
        return DegreeCentrality(new_G , node)
    else:
        raise("Metric Not Allowed")

In [21]:
BestUser(combined_graph , 31505 , '2008/09/01', '2008/09/30', 'ClosenessCentrality')

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

# Exercise 3

In [ ]:
def ShortOrferedRoute (G , users , source , target):
    total_dist = 0
    full_path = [source]
    users_bis = [source]+users+[target]
      
    for i in range ( len(users_bis) -1) :
        dist , path = Dijkstra(G,users_bis[i],users_bis[i+1]) 
        if dist == "Disconnected" :
            return "Disconnected"
        total_dist += dist
        full_path += path[1:]
    full_path = [int(i) for i in full_path ]
    return total_dist , full_path

In [ ]:
ShortOrferedRoute ( combined_graph , [ 124 , 285 ] , 624,1238)

# Exercise 4

In [ ]:
def DisconnectingGraph (G , df , start_1 , end_1 , start_2 , end_2) :
    list_pairs_graph1 = []
    list_pairs_graph2 = []
    links_graph1 = 0
    weight_graph1 = 0
    links_graph2 = 0
    weight_graph2 = 0
    
    
    Graph1 = nx.DiGraph()
    Graph2 = nx.DiGraph()
    
    #Convert the date to timestamp
    s_tstmp1 = convert_date(start_1)
    e_tstmp1 = convert_date(end_1)
    s_tstmp2 = convert_date(start_2)
    e_tstmp2 = convert_date(end_2)
    
    
    #Create two dataframes containing only the interaction in the intereset nteval
    filtered_1 = filterGraph( df , s_tstmp1 , e_tstmp1 )
    filtered_2 = filterGraph( df , s_tstmp2 , e_tstmp2 )
    
    
    #Create a weighted graph
    filtered_1.apply(lambda x :graphWeighter( Graph1 , x) , axis=1)
    filtered_2.apply(lambda x :graphWeighter( Graph2 , x) , axis=1)
    
    
    #Find users
    
    user_1 = findUsers(Graph1 , Graph2)
    user_2 = findUsers(Graph2 , Graph1)
    
    #Find all nodes deinited in both the graphs
    nodes_both_graph = []
    for node in list(Graph1.nodes):
        if node in list(Graph2.nodes):
            nodes_both_graph.append(node)
        
        
    for element in nodes_both_graph :
        for _ , checkPoint , attr in Graph1.edges(element , data = True):
            list_pairs_graph1.append((element , checkPoint))
            links_graph1 +=1
            weight_graph1 += attr['weight']
    
    for element in nodes_both_graph :
        for _ , checkPoint , attr in Graph2.edges(element , data = True):
            list_pairs_graph2.append((element , checkPoint))
            links_graph2 +=1
            weight_graph2 += attr['weight']
    
    
    if (links_graph1 < links_graph2) :
        return Graph1, Graph2 , nodes_both_graph , list_pairs_graph1
    else :
        return Graph1, Graph2 , nodes_both_graph , list_pairs_graph2

In [ ]:
def findUsers (G1, G2):
    for node in list(G1.nodes):
        if node not in G2.nodes:
            return node
    return ValueError("Graphs are defined on the same nodes")

In [ ]:
def frontEnd1 (Graph , start_time , end_time) :
    s_tstmp = convert_date (start_time) 
    e_tstmp = convert_date (end_time)
    
    #edges in the graph during the time interval
    edges = [(s , t , attr) for s,t,attr in Graph.edges(data = True) if s_tstmp < attr['tstmp'][0] < e_tstmp]
    
    H = nx.DiGraph()
    H.add_edges_from(edges)
    overall_features = GraphDescription(Graph)
    
    df = pd.DataFrame(overall_features , ["NumberOfUsers", "NumberOfAnswers/Comments", "Directed", 
                                         "AverageLinksPerUser", "DensityDegree", "Sparse/Dense"])
    
    df.columns = ['OverallFeatures']
    
    display(df)
    
    #plot degree distribution
    
    degree = [degree_calculator(Graph , node) for node in list(Graph.nodes())]
    plt.title('Digree Distribution')
    plt.hist(degree)
    plt.show()

In [ ]:
frontEnd1(combined_graph, '2008/08/01', '2008/08/15')

In [ ]:
def split_equally(start_date, end_date, intervals):
    test_date1 = datetime.datetime.strptime(start_date, '%Y/%m/%d')
    test_date2 = datetime.datetime.strptime(end_date, '%Y/%m/%d')
    temp =[]
    diff = (test_date2 - test_date1) // intervals
    for idx in range(0 , intervals) :
        temp.append(test_date1 + idx * diff)
        
    res = []
    
    for sub in temp :
        res.append(sub.strftime("%Y/%m/%d") )
    res.append(end_date)
    return res
    
    
def frontEnd2(Graph , interest_node , start_time , end_time , metric):
    plot_nodes = [interest_node] + list(Graph.neighbors(interest_node))
    plot_nodes = [int(i) for i in plot_nodes]
    score = []
    
    H = Graph.subgraph(plot_nodes)
    
    plt.figure(figsize=(10 , 5 ))
    plt.figure(1)
    plt.title("Node {} and its neighbors".format(interest_node))
    color_map = ['#9e2a2b' if node == interest_node else '#e09f3e' for node in H]
    pos=nx.spring_layout(H,scale=2)
    nx.draw(H, with_labels = True, node_color=color_map,pos = pos, alpha = 0.7)
    
    
    interval_number = 5
    intervals = split_equally(start_time,end_time, interval_number)
    for i in range(interval_number):
        start = intervals[i]
        end = intervals[i+1]
        try :
            score.append( BestUser(H ,  interest_node , start , end , metric))
        except :
            score.append(0)
        
    #print("score : {}".format(score))
    label_x = [f'{[intervals[i],intervals[i+1]]}' for i in range(len(intervals)-1)]
    
    plt.figure(2)
    plt.title('Score during time')
    
    plt.xticks(rotation=45, ha='right')
    plt.bar(label_x, score, color = '#335c67')

In [ ]:
frontEnd2(combined_graph, 31505, '2008/09/01', '2008/09/30', 'DegreeCentrality')

In [ ]:
def fronEnd3(G, users , source , target ):
    short_path = ShortOrferedRoute(G , users , source , target)
    
    H = nx.DiGraph()
    
    route = short_path[1]
    for i in range(len(route)-1) :
        H.add_edge(route[i] , route[i+1])
    
    layout = nx.spring_layout(H,scale=2)
    nx.draw_networkx_nodes (H , pos = layout , nodelist = [source] , node_color = "#9e2a2b" , label = "Initial User of path",
                           node_size = 100 , alpha = 0.5)
    nx.draw_networkx_nodes (H , pos = layout , nodelist = [target] , node_color = "#335c67" , label = "End User of path",
                           node_size = 100 , alpha = 0.5)
    nx.draw_networkx_nodes (H , pos = layout , nodelist = users , node_color = "#e09f3e" , label = "Constrain Users of path",
                           node_size = 100 , alpha = 0.5)
    nx.draw_networkx_nodes (H , pos = layout , nodelist = route , node_color = '#fff3b0' , label = "Users",
                           node_size = 100 , alpha = 0.5)
    plt.tight_layout()
    plt.legend(scatterpoints = 1 , markerscale = 0.4)
    color_map = ['#9e2a2b' if node == source else '#335c67' if node == target else '#e09f3e' if node in users else '#fff3b0' for node in H]
    
    nx.draw(H , with_labels = True , node_color = color_map , alpha = 0.7 , pos = layout , arrowsize = 20 , connectionstyle='arc3,rad=0.05', node_size = 1000)

In [ ]:
fronEnd3(combined_graph,[2,5],1,8)

In [ ]:
MainGraph, SecondGraph, list_intersection, list_couples = DisconnectingGraph(combined_graph ,  atq , '2008/08/01', '2008/08/10' ,'2008/09/01', '2008/09/10'  )

plt.figure(figsize = (20 , 10))
plt.subplot(121)
nx.draw(MainGraph , with_labels = True , edge_color= "grey" )

plt.subplot(122)
nx.draw(SecondGraph , with_labels=True , node_color = 'green' , edge_color = 'green')

plt.figure(figsize = (20,20))
mergeGraph = nx.compose(MainGraph , SecondGraph)
pos = nx.spring_layout(mergeGraph)
nx.draw(mergeGraph , with_labels=True , pos = pos , edge_color = 'blue')
nx.draw_networkx_edges(mergeGraph , pos = pos , edgelist=SecondGraph.edges(data=True) , edge_color='green')

nx.draw_networkx_nodes(mergeGraph , nodelist= list_intersection , node_color = '#D90368', pos=pos, node_size=1000)
nx.draw_networkx_edges(mergeGraph, edgelist=list_couples, width=5.0,  pos=pos, edge_color='#D90368')
plt.show()